In [1]:
import torch
import torch.nn as nn
import timm

from LO import Loader
from TR import Trainer
from TE import Tester

import copy
from FE import FeatureExtractor
from torch.utils.data import TensorDataset, DataLoader
from R import RGenerator
import pickle

timm_models = [
    "resnet18.a1_in1k",
    "resnet152.a1_in1k",
    "densenet201.tv_in1k",
    "resnext50_32x4d.ra_in1k",
    "efficientnet_b0.ra_in1k",
    "vit_small_patch16_224.dino",
    "vit_base_patch16_224.dino",
    "vit_base_patch32_224.sam_in1k",
    "vit_base_patch16_224.sam_in1k"
]

timm_models = ["vit_base_patch14_dinov2.lvd142m"]

sample_size = 518
root_dir = "/home/stagiaire/D/R/patchs/70"
cross_validations = 5
num_folds = 5
batch_size = 16

for model_name in timm_models:
    model = timm.create_model(model_name, pretrained=True, num_classes=0)
    num_features = model(torch.randn(1, 3, sample_size, sample_size)).shape[1]
    head = nn.Sequential(
        nn.Linear(num_features, 1000),
        nn.ReLU(),
        nn.Linear(1000, 1000),
        nn.ReLU(),
        nn.Linear(1000, 2)
    )

    metrics = [[] for _ in range(cross_validations)]

    for n in range(cross_validations):
        processor = Loader(root_dir=root_dir, num_folds=num_folds)
        processor.load_data(sample_size=sample_size, batch_size=batch_size)
        loaders = processor.loaders

        fe = FeatureExtractor(model_name)
        floaders = []
        for loader in loaders:
            features, labels = fe.get_features(loader)
            dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels))
            floaders.append(DataLoader(dataset, batch_size=batch_size, shuffle=True))

        for i in range(num_folds):
            head_i = copy.deepcopy(head)

            train_loaders = [floaders[(j + i) % num_folds] for j in range(num_folds - 1)]
            test_loader = floaders[(i + num_folds - 1) % num_folds]

            for train_loader in train_loaders:
                trainer = Trainer(head_i, train_loader)
                trainer.train(num_epochs=40)

            tester = Tester(head_i, test_loader, processor.dataset)
            tester.evaluate()

            metrics[n].append(tester.report)

    with open(f"/home/stagiaire/D/B/metrics/{model_name}.pkl", 'wb') as f:
        pickle.dump(metrics, f)

    new_metrics = []
    for l in metrics:
        new_metrics.extend(l)
    print(f"{model_name}")
    reporter = RGenerator(new_metrics).report()


/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Training: 100%|████████████████████████████| 40/40 [00:00<00:00, 53.29 epochs/s]



Jaccard index:  56.8%


                Predicted Class 0  Predicted Class 1
Actual Class 0                128                 58
Actual Class 1                 47                138


              precision    recall  f1-score   support

           0       0.73      0.69      0.71       186
           1       0.70      0.75      0.72       185

    accuracy                           0.72       371
   macro avg       0.72      0.72      0.72       371
weighted avg       0.72      0.72      0.72       371




Training: 100%|████████████████████████████| 40/40 [00:00<00:00, 62.06 epochs/s]



Jaccard index:  60.6%


                Predicted Class 0  Predicted Class 1
Actual Class 0                126                 53
Actual Class 1                 44                149


              precision    recall  f1-score   support

           0       0.74      0.70      0.72       179
           1       0.74      0.77      0.75       193

    accuracy                           0.74       372
   macro avg       0.74      0.74      0.74       372
weighted avg       0.74      0.74      0.74       372




Training: 100%|████████████████████████████| 40/40 [00:00<00:00, 57.11 epochs/s]


...


Jaccard index:  53.8%


                Predicted Class 0  Predicted Class 1
Actual Class 0                137                 54
Actual Class 1                 54                126


              precision    recall  f1-score   support

           0       0.72      0.72      0.72       191
           1       0.70      0.70      0.70       180

    accuracy                           0.71       371
   macro avg       0.71      0.71      0.71       371
weighted avg       0.71      0.71      0.71       371




Training: 100%|████████████████████████████| 40/40 [00:00<00:00, 60.46 epochs/s]



Jaccard index:  57.8%


                Predicted Class 0  Predicted Class 1
Actual Class 0                113                 56
Actual Class 1                 53                149


              precision    recall  f1-score   support

           0       0.68      0.67      0.67       169
           1       0.73      0.74      0.73       202

    accuracy                           0.71       371
   macro avg       0.70      0.70      0.70       371
weighted avg       0.71      0.71      0.71       371




Training: 100%|████████████████████████████| 40/40 [00:00<00:00, 61.92 epochs/s]



Jaccard index:  50.4%


                Predicted Class 0  Predicted Class 1
Actual Class 0                125                 47
Actual Class 1                 75                124


              precision    recall  f1-score   support

           0       0.62      0.73      0.67       172
           1       0.73      0.62      0.67       199

    accuracy                           0.67       371
   macro avg       0.68      0.67      0.67       371
weighted avg       0.68      0.67      0.67       371


vit_base_patch14_dinov2.lvd142m
Precision (mean, stdev): 71%, 3%
Recall (mean, stdev): 75%, 6%
F1-score (mean, stdev): 72%, 3%
IoU (mean, stdev): 57%, 4%

[[127.  58.]
 [ 46. 139.]]

[[13. 10.]
 [11. 14.]]

